In [2]:
%pip install fastparquet

import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Parameters
data = "../../salsa/data/single_partition/partition_0/left_index.gzip"
sample = 500
number_of_interactions = 1
output = "../data/users.json"

In [4]:
# Import users from dataset
keep_columns = [0, 1]

df = pd.read_parquet(data, columns=['adjacency_list', 'user_id'])
df.reset_index(level=0, inplace=True)

In [5]:
df["len_adjacency_list"] = df["adjacency_list"].str.len()
active_users = df[df["len_adjacency_list"] > number_of_interactions]["user_id"]

,user_id,adjacency_list,len_adjacency_list
0,12,"[1367460913012346880, 1367460911875694601, 136...",73
2,20,"[1365021188880142343, 1364471391118970882, 135...",25
24,785,"[1368147812542603268, 1368107699125198849, 136...",26
31,1186,"[1366834300310548483, 1367525263639072770, 136...",28
33,1378,"[1367275474737594369, 1367600777485836289, 136...",25
...,...,...,...
3302439,1368428098597756930,"[1357927702469046273, 1365897106007891970, 136...",33
3302541,1368459012379996162,"[1368637571404816385, 1368639072026132481, 136...",12
3302733,1368514221969842177,"[1366297151710457861, 1366277019093377024, 136...",12
3302941,1368570705114247170,"[1366028710655090694, 1366054008738230273, 136...",11


In [5]:
# Sample users
if sample:
    active_users = active_users.sample(n=sample)

In [6]:
# Save results
active_users.to_json(output, orient="table", index=False)
active_users.reset_index(drop=True)

0       836161636041961472
1                417920581
2      1322530154577420288
3               3104601972
4                569581606
              ...         
495              131362254
496             2414744485
497    1236667536210571264
498             1851527372
499    1271951093606625284
Name: user_id, Length: 500, dtype: int64